In [209]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Ensure you have your dataset loaded into variables X (features) and y (labels)
# X and y should be numpy arrays or pandas dataframes/series


In [210]:
# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(device)

mps


In [211]:
import pandas as pd
# Read the Excel file into a DataFrame
df = pd.read_csv('data_v2.csv')

In [140]:
import numpy as np
for column in df.columns:
    for index, value in df[column].items():
        if value == -1000:
            df.at[index, column] = np.random.randint(1, 361)

In [212]:
print(df)

         Asc  Sun  Moon  Mercury  Venus  Mars  Jupiter  Saturn  Rahu  Chubby  \
0          0  227     0      357    160    12      203     320   128       1   
1          0  171     0      306    148   148       14      61    14       1   
2          0   25     0       49    176    17      316       2   147       1   
3          0  210     0      247    217   256       44     355     0       1   
4          0  359     0      268    284    34      236     117   321       1   
...      ...  ...   ...      ...    ...   ...      ...     ...   ...     ...   
5631455   58  315    60      297     93   204      140     182   360       0   
5631456  218  284    60      245    269    65      241     202   360       0   
5631457  162  216    60      208     19   188        9      19   360       0   
5631458   25  357    60      184    325    69      285     189   360       0   
5631459  356  109    60      333     96   189       53     269   360       0   

                                       

In [213]:
# Separate features (X) and target values (y)
X = df.iloc[:, :-2]  # Assuming columns A to N are the features
y = df.iloc[:, -2]   # Assuming column O is the target

# Convert to NumPy arrays
X = X.to_numpy()
y = y.to_numpy()

In [214]:
# Assume X and y are numpy arrays or pandas DataFrames/Series
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.int).to(device)  # Ensure y is 2D for regression
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.int).to(device)

# Create DataLoader for training data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Create DataLoader for test data
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [215]:
print(X_train_tensor)

tensor([[ 1.4236, -0.5375,  0.8253,  ..., -1.3947,  1.2056,  1.0272],
        [ 0.7906,  0.4223, -0.6103,  ..., -0.7821,  1.0711,  0.0564],
        [ 1.6922,  0.5087, -1.4652,  ..., -1.5383, -0.5337, -0.9720],
        ...,
        [-0.7153, -1.2477,  1.1841,  ...,  0.6632,  1.7245, -0.3184],
        [ 0.7810,  0.3167,  0.2025,  ...,  0.0315,  1.5708,  0.5755],
        [-1.2237,  0.1727,  0.9519,  ..., -1.1267,  0.1005,  0.0180]],
       device='mps:0')


In [177]:
# class Classifier(nn.Module):
#     def __init__(self, input_dim):
#         super(Classifier, self).__init__()
#         self.fc1 = nn.Linear(input_dim, 64)
#         self.fc2 = nn.Linear(64, 128)
#         self.fc3 = nn.Linear(128, 128)
#         self.fc4 = nn.Linear(128, 64)
#         self.fc5 = nn.Linear(64, 1)
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(0.7)

#     def forward(self, x):
#         x = self.relu(self.fc1(x))
#         x = self.dropout(x)  # Apply dropout
#         x = self.relu(self.fc2(x))
#         x = self.relu(self.fc3(x))
#         x = self.relu(self.fc4(x))
#         x = self.fc5(x)
#         return x
# print(X_train.shape[1])
# input_dim = X_train.shape[1]
# model = Classifier(input_dim).to(device)  # Move the model to the GPU

# # Initialize weights
# def weights_init(m):
#     if isinstance(m, nn.Linear):
#         nn.init.xavier_uniform_(m.weight)
#         nn.init.zeros_(m.bias)

# model.apply(weights_init)


18


Classifier(
  (fc1): Linear(in_features=18, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.7, inplace=False)
)

In [221]:
class Classifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.relu(out)
        out = self.fc4(out)
        return out
    
# Initialize the classifier model
input_size = X.shape[1]
hidden_size = 64
num_classes = 2
model = Classifier(input_size, hidden_size, num_classes).to(device)

In [222]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")


Epoch 1/500, Loss: 48554.43394035101
Epoch 2/500, Loss: 48723.54123657942
Epoch 3/500, Loss: 48723.240082740784


KeyboardInterrupt: 

In [218]:
torch.save(model.state_dict(), 'my_model.pth')

In [ ]:
# Evaluation
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total}%")

In [220]:
from torch import arange


# Function to make predictions on new user input
def predict(model, user_input):
    # Ensure the model is in evaluation mode
    model.eval()
    with torch.no_grad():
        # Standardize the user input using the same scaler
        user_input = scaler.transform(user_input.reshape(1, -1))
        # Convert user input to a tensor
        user_input_tensor = torch.tensor(user_input, dtype=torch.float32)
        # Reshape to match the input size
        user_input_tensor = user_input_tensor.to(device)
        # Pass through the model
        output = model(user_input_tensor)
        # Get the predicted class
        _, predicted_class = torch.max(output, 1)
    return predicted_class.item()

model.eval()
input = np.array([263.44, 22.29, 358.99,5.03, 48.50,263.44,358.73,224.26,15])
output = predict(model,input)
print(f"Predicted class for the user input: {output}")

for moon in arange(0,361):
    model.eval()
    input[2] = moon
    output = predict(model,input)
    print(output)






Predicted class for the user input: 1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
